## Reviews from IMDb
E.g. the movie 'The Substance' has a url-page 'https://www.imdb.com/title/tt17526714/' where 'tt17526714' is the ImdbId.

In [1]:
# !pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
import requests
from bs4 import BeautifulSoup

def scrapeReviews(ImdbId, sort="submissionDate", ratingFilter=10, dir="asc") :
    
    movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews/_ajax?"+"sort="+sort+"&dir="+dir+"&ratingFilter="+str(ratingFilter)
    # print(movie_url)
    r = requests.get(url=movie_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # print(soup.prettify())
    try :
        reviews = soup.find_all('div', {'class' : 'imdb-user-review'})
        # print(reviews)
        # return
    except:
        raise Exception("Error in reviews")
        
    
    data = {}
    for review in reviews :
        # print(BeautifulSoup(review.text, 'html.parser').prettify())
        # print(review)
        review_imdb={}
        
        try :
            review_imdb['reviewer_name']=review.find('span',{'class':'display-name-link'}).find('a').string.strip()
        except :
            raise Exception("Error in reviewer_name")
        
        try:
            review_imdb['reviewer_url']=review.find('span',{'class':'display-name-link'}).find('a')['href']
        except:
            raise Exception("Error in reviewer_url")
            
        try:
            short_review =review.find('a',{'class': 'title'})
            text=short_review.string.strip()
            review_imdb['title']=text
        except:
            raise Exception("Error in title")
    
        full_review = review.find('div',{'class' : 'show-more__control'})
        # print(full_review)  
        text = full_review.get_text(strip=True) # Extract text from it
        # print(text)
        review_imdb['full_review']=text
        
        data[review['data-review-id']] = review_imdb
        break
    
    return data

def extractReviews(ImdbId):
    data = scrapeReviews(ImdbId)
    return data
extractReviews('tt17526714')  # Reviews for the substance

{'rw9917701': {'reviewer_name': 'piszczatowski-mateusz',
  'reviewer_url': '/user/ur138769786/',
  'title': 'ABSOLUTELY INSANE',
  'full_review': 'I was extremely hyped for that movie even though I\'m not a big fan of Fargeat\'s debut film "Revenge". I liked her as a director, but not as a screenwriter. While that movie had interesting visual style, the screenplay was bad & stupid and the characters were uninteresting.However she improved a lot from her last movie. "The Substance" looks amazing, the shots, the score, the editing, production design and even costumes are well thought out. The screenplay is also great (she actually won an award for Best Screenplay at Cannes for "The Substance"), the decisions that characters are making are understandable - they are results from characters\' determination to achieve their goals.I also loved Demi Moore\'s perfomance (it\'s worth mentioning that I wasn\'t considering her good actress at all, but after watching that movie, the first thought t

MINOR CHANGES TO COPY FROM https://github.com/pavan412kalyan/imdb-movie-scraper/blob/main/api/reviews.py

In [24]:
# -*- coding: utf-8 -*-

# this is for scraping reviews given imdb id - also with filters from api urls

import requests
from bs4 import BeautifulSoup
import re
import json
def scrapeReviews(ImdbId,sort="submissionDate",ratingFilter=10,dir="asc") :
    
    try :
#        movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews?"+"sort="+sort+"&dir="+dir+"&ratingFilter="+ratingFilter
         movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews/_ajax?"+"sort="+sort+"&dir="+dir+"&ratingFilter="+ratingFilter
         print(movie_url)
    except :
        
#        movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews"
         movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews"+"/_ajax"
        
    r = requests.get(headers={'User-Agent': 'Mozilla/5.0'},url=movie_url)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'html.parser')
    try :
        reviews = soup.find_all('div',{'class' : 'imdb-user-review'})
    except :
        pass
        
    
    #review = soup.find('div',{'class' : 'imdb-user-review'})
    
    data = {}
    data['ImdbId'] = ImdbId
    reviews_text =[]
    for review in reviews :
        review_imdb={}
        
        ################
        try :
            review_imdb['reviewer_name']=review.find('span',{'class':'display-name-link'}).find('a').string.strip()
        except :
            review_imdb['reviewer_name']=""    
        ###############
        try :
            review_imdb['reviewer_url']=review.find('span',{'class':'display-name-link'}).find('a')['href']
        except:
            review_imdb['reviewer_url']=""
        ############
        try :
            review_imdb['data-review-id']=review['data-review-id']
        except :
            review_imdb['data-review-id']=""
            
        #############
        try:
            short_review =review.find('a',{'class': 'title'})
            text=short_review.string.strip()
            review_imdb['short_review']=text
        except :
            review_imdb['short_review']=""
    
        ######################
        try :
            full_review = review.find('div',{'class' : 'show-more__control'})
            text = full_review.string.strip()
            review_imdb['full_review']=text
        except :
             review_imdb['full_review']=""
        #############
        try :
            review_date = review.find('span',{'class' : 'review-date'})
            text=review_date.string.strip()
            review_imdb['review_date'] =text    
        except :
             review_imdb['review_date']  ="" 
        #######
        try :
            ratings_span = review.find('span',{'class' : 'rating-other-user-rating'})
            text=ratings_span.find('span').string.strip()
            review_imdb['rating_value']  = text      
        except :
            review_imdb['rating_value']  = "" 
        ##########
        reviews_text.append(review_imdb)    
    data['reviews']=reviews_text
    return data

scrapeReviews('tt17526714')

{'ImdbId': 'tt17526714',
 'reviews': [{'reviewer_name': 'Sugarfreee',
   'reviewer_url': '/user/ur187764144/?ref_=tt_urv',
   'data-review-id': 'rw10016456',
   'short_review': 'A spiritual successor to Cronenberg style body horror',
   'full_review': '',
   'review_date': '17 September 2024',
   'rating_value': '8'},
  {'reviewer_name': 'maria_nawrocka',
   'reviewer_url': '/user/ur49877390/?ref_=tt_urv',
   'data-review-id': 'rw9806680',
   'short_review': 'Breasts just in the right place',
   'full_review': '',
   'review_date': '24 May 2024',
   'rating_value': '8'},
  {'reviewer_name': 'julie-ford',
   'reviewer_url': '/user/ur18692252/?ref_=tt_urv',
   'data-review-id': 'rw10000136',
   'short_review': 'Future cult classic',
   'full_review': '',
   'review_date': '7 September 2024',
   'rating_value': '9'},
  {'reviewer_name': 'FeastMode',
   'reviewer_url': '/user/ur104603847/?ref_=tt_urv',
   'data-review-id': 'rw10020012',
   'short_review': 'My single most memorable movie ex